In [65]:
# install packages


# import packages

import pandas as pd
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

#hmm
from hmmlearn import hmm



# format

# def stock_data(stock_name -> str):

# Return train_data, test_data 


# def  model_function(train_data -> np.array, num_of_predictions -> int):

# 	Return prediction 


# def evaluation(test_data -> np.array, prediction -> np.array):
	
# 	Return metrics


# pick stocks

In [66]:
# load and group by cap size
stocks_screener = pd.read_csv('nasdaq_screener_1637428146284.csv') #https://www.nasdaq.com/market-activity/stocks/screener
stocks_screener = stocks_screener[stocks_screener['Market Cap'].notna()]

M = 10**6
B = 10**9

bins =  np.array([0,50*M,300*M,2*B,10*B,200*B,np.inf])
ind = np.digitize(stocks_screener['Market Cap'], bins)

stocks_screener['Cap Group'] = ind

gb = stocks_screener.groupby('Cap Group')
list_of_groupbed_stocks = [gb.get_group(x) for x in gb.groups]

cap_list = ['nano','micro','small','medium','large','mega']
stock_cap_dict = dict()
for i in range(len(cap_list)):
    stock_cap_dict[cap_list[i]] = list_of_groupbed_stocks[i]['Symbol']
    


In [67]:
# def stock_picker(stock_dict = stock_cap_dict,cap = 'small',num_stocks = 100):
    
#     ticker_list = list((stock_dict[cap]).sample(n=num_stocks))

#     return ticker_list

In [68]:
# sample = stock_picker(cap = 'large',num_stocks = 10)
# sample

In [69]:
# def ticker_to_data(ticker = 'aapl',want_log = True):
#     stock_data = yf.download(ticker, start="2018-01-01", end="2021-12-31").reset_index()
#     stock_data['daily_return'] = (stock_data.Close /
#         stock_data.Close.shift(-1)).dropna()
    
#     stock_data['Log_daily_return'] = np.log(stock_data['daily_return'])
    
#     train_full, test_full = stock_data[stock_data.Date<'2021-01-01'],stock_data[stock_data.Date>='2021-01-01']
    
#     print(ticker)
#     print(len(train_full))
#     print(len(test_full))
#     if want_log:    
#         train_close,test_close = np.array(train_full.Log_daily_return),np.array(test_full.Log_daily_return)
#     else:
#         train_close,test_close = np.array(train_full.daily_return),np.array(test_full.daily_return)
#     return train_close,test_close,stock_data

In [79]:
def pick_stock_get_data(stock_dict = stock_cap_dict,cap = 'small',num_stocks = 100,want_log = True,start_date="2018-01-01",
                        end_date="2021-12-31",cut_date = '2021-01-01' ):
    
    #length standards
    stock_data = yf.download('AAPL', start=start_date, end=end_date,progress=False).reset_index()
    standard_length = len(stock_data)
    
    count = 0
    
    resample_count = 0
    
    train_data = {}
    test_data = {}
    
    while count<num_stocks:
        
        length = 0
        
        resample_count-=1
        
        while length!=standard_length:
            resample_count+=1
            stock_sample =  list((stock_dict[cap]).sample(n=1))[0]
            stock_data = yf.download(stock_sample, start=start_date, end=end_date,progress = False).reset_index()
            length = len(stock_data)
        
        
        stock_data['daily_return'] = (stock_data.Close /stock_data.Close.shift(-1)).dropna()
        
        stock_data['Log_daily_return'] = np.log(stock_data.Close /stock_data.Close.shift(-1).dropna())
        
        train_full, test_full = stock_data[stock_data.Date<cut_date],stock_data[stock_data.Date>=cut_date]
        
        if want_log:
        
            train_data[stock_sample],test_data[stock_sample] = np.array(train_full.Log_daily_return),np.array(test_full.Log_daily_return)
            
        else:
            
            train_data[stock_sample],test_data[stock_sample] = np.array(train_full.daily_return),np.array(test_full.daily_return)
        
        count+=1
        
    print(resample_count)
    return train_data,test_data

# evaluation

In [ ]:
def evaluation_metrics(test_data, predictions):
    metrics = {}
    
    metrics['MAPE'] = mean_absolute_percentage_error(test_data,predictions)
    metrics['MSE'] = mean_squared_error(test_data,predictions)
    
    metrics['C']

# Models

In [ ]:
def hmm_prediction(train_data,num_of_predictions):
    hmm_model = hmm.GaussianHMM(n_components=3, covariance_type="full", n_iter=1000)
    hmm_model.fit(train_data)
    
    predict_state = hmm_model.predict(train_data)
    
    hmm_prediction = hmm_model.sample(num_of_predictions,random_state = predict_state[-1])
    hmm_prediction_value = hmm_prediction[0]
    hmm_prediction_states = hmm_prediction[1]
    
    return hmm_prediction_value